In [91]:
import os
import json
import pandas as pd
import json

In [92]:
from langchain.chat_models import ChatOpenAI

In [93]:
from dotenv import load_dotenv
load_dotenv()

True

In [94]:
KEY=os.getenv("OPENAI_API_KEY")

In [95]:
llm=ChatOpenAI(openai_api_key=KEY, model_name='gpt-3.5-turbo', temperature = 0.5)

In [96]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [127]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2":
        {
            "three_marks_questions": "three_marks_questions",
            "type": "short_answer",
            "question": "Three marks question here"
        },
    "3": 
        {
            "five_marks_questions": "five_marks_questions",
            "type": "long_answer",
            "question": "Five marks question here"
        },
}

In [128]:
TEMPLATE="""
Text: {text}
You are an expert in creating MCQs and detailed answer questions. Given the above text, your task is to create a quiz comprising {mcq_number} multiple choice questions (MCQs), {three_marks_number} questions worth 3 marks, and {five_marks_number} questions worth 5 marks for {subject} students. Maintain a {tone} tone throughout the quiz. 
Ensure the questions are unique, relevant to the provided text, and that they accurately assess the students' understanding of the subject matter.
Adhere to the following format in your RESPONSE_JSON below as a guide. Create {mcq_number} MCQs, {three_marks_number} questions for 3 marks, and {five_marks_number} questions for 5 marks, ensuring a comprehensive evaluation.

{response_json}
"""

In [129]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "mcq_number", "three_marks_number", "five_marks_number", "subject", "tone", "response_json"],
    template=TEMPLATE
)

In [130]:
quiz_chain = LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)


In [131]:
TEMPLATE2 = """
You are an expert English grammarian and writer. Given a comprehensive quiz for {subject} students, including multiple choice questions (MCQs), questions worth 3 marks, and questions worth 5 marks, your task is to evaluate the overall complexity of the quiz. Provide a succinct complexity analysis, not exceeding 50 words. Assess whether the quiz aligns with the cognitive and analytical abilities of the intended student audience. 

If you find any aspects of the quiz not up to par—whether it be the content, cognitive demands, or tone—identify these issues specifically. Suggest necessary modifications to ensure the quiz questions are appropriately challenging and engaging, and adjust the tone to perfectly match the students' abilities.

Content:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [132]:
quiz_evaluation_prompt = PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE2)

In [133]:
review_chain = LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [135]:
generate_evaluate_chain = SequentialChain(
    chains=[quiz_chain, review_chain],
    input_variables=["text", "mcq_number", "three_marks_number", "five_marks_number", "subject", "tone", "response_json"],
    output_variables=["quiz", "review"],
    verbose=True
)

In [136]:
file_path=r"C:\Projects\OpenAI\mcqgen\data.txt"

In [137]:
with open(file_path, 'r') as file:
    TEXT = file.read()

In [138]:
print(TEXT)

Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]

Biologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each defined by the nature of their research questions and the tools that they use.[7][8][9] Like other scientists, bio

In [139]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"three_marks_questions": "three_marks_questions", "type": "short_answer", "question": "Three marks question here"}, "3": {"five_marks_questions": "five_marks_questions", "type": "long_answer", "question": "Five marks question here"}}'

In [140]:
MCQ_NUMBER=5 
THREE_MARKS_NUMBER = 5
FIVE_MARKS_NUMBER = 5
SUBJECT="biology"
TONE="simple"

In [141]:
with get_openai_callback() as cb:
    response = generate_evaluate_chain(
        {
            "text": TEXT,
            "mcq_number": MCQ_NUMBER,
            "three_marks_number": THREE_MARKS_NUMBER,
            "five_marks_number": FIVE_MARKS_NUMBER,
            "subject": SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
    )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text: Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]

Biologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each defin

In [142]:

print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:2358
Prompt Tokens:1504
Completion Tokens:854
Total Cost:0.0039640000000000005


In [143]:
response

{'text': 'Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]\n\nBiologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each defined by the nature of their research questions and the tools that they use.[7][8][9] Like other sci

In [144]:
quiz_data_json = response['quiz']

In [145]:
quiz_data = json.loads(quiz_data_json)

In [146]:
quiz_table_data = []

# Process multiple choice questions
for key, value in quiz_data.items():
    if "mcq" in value:
        mcq = value["mcq"]
        options = " | ".join([f"{option}: {option_value}" for option, option_value in value["options"].items()])
        correct = value["correct"]
        quiz_table_data.append({"Question Type": "MCQ", "Question": mcq, "Options": options, "Correct Answer": correct})

# Process three marks questions
for key, value in quiz_data.items():
    if "three_marks_questions" in value:
        question = value["three_marks_questions"]
        quiz_table_data.append({"Question Type": "Three Marks", "Question": question})

# Process five marks questions
for key, value in quiz_data.items():
    if "five_marks_questions" in value:
        question = value["five_marks_questions"]
        quiz_table_data.append({"Question Type": "Five Marks", "Question": question})

print(quiz_table_data)


[{'Question Type': 'MCQ', 'Question': 'Which of the following is a unifying theme in biology?', 'Options': 'a: Evolution | b: Chemistry | c: Physics | d: Geology', 'Correct Answer': 'a'}, {'Question Type': 'MCQ', 'Question': 'What is the importance of energy processing in living organisms?', 'Options': 'a: It allows organisms to communicate | b: It helps organisms regulate their internal environments | c: It enables organisms to move, grow, and reproduce | d: It controls the genetic information in organisms', 'Correct Answer': 'c'}, {'Question Type': 'MCQ', 'Question': 'Which of the following is not a eukaryotic organism?', 'Options': 'a: Protist | b: Bacteria | c: Plant | d: Fungi', 'Correct Answer': 'b'}, {'Question Type': 'MCQ', 'Question': 'What is the main tool that biologists use to study life?', 'Options': 'a: Microscope | b: Calculator | c: Telescope | d: Compass', 'Correct Answer': 'a'}, {'Question Type': 'MCQ', 'Question': 'Which level of organization in biology involves the 

In [148]:
df_questions = pd.DataFrame(quiz_table_data)

# Print the DataFrame to see the output
print(df_questions)

# Save the DataFrame to an Excel file
df_questions.to_excel('quiz_questions.xlsx', index=False)

   Question Type                                           Question  \
0            MCQ  Which of the following is a unifying theme in ...   
1            MCQ  What is the importance of energy processing in...   
2            MCQ  Which of the following is not a eukaryotic org...   
3            MCQ  What is the main tool that biologists use to s...   
4            MCQ  Which level of organization in biology involve...   
5    Three Marks  Explain the significance of evolution in the u...   
6    Three Marks  Describe the process of energy processing in l...   
7    Three Marks  Discuss the role of cells in processing heredi...   
8    Three Marks  Explain the concept of biodiversity and its im...   
9    Three Marks  Describe how biologists use the scientific met...   
10    Five Marks  Discuss the different subdisciplines within bi...   
11    Five Marks  Explain the process of classification of organ...   
12    Five Marks  Describe the diversity of life on Earth and th...   
13    

In [150]:
from datetime import datetime
datetime.now().strftime('%m_%d_%Y_%H_%M_%S')

'03_28_2024_18_53_00'